In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from gensim import corpora
lemmatizer = WordNetLemmatizer()
nltk.download('stopwords')
da_stop = set(nltk.corpus.stopwords.words('danish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eskeh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
sent_df = pd.read_csv("2_headword_headword_polarity.csv", header=None)[[0, 4]]
sentiment_dict = {row[0]: row[4] for _, row in sent_df.iterrows()}

def get_tfs(text):
    return dict(nltk.probability.FreqDist(([lemmatizer.lemmatize(w) for word in text if (w:=word.lower()).isalpha()])))

def get_sents(text):
    tf = get_tfs(nltk.word_tokenize(text))
    return (
        np.sum([sentiment_dict[word] * tf[word] for word in tf.keys() if word in list(sentiment_dict.keys())]) / 
        np.sum([tf[word] for word in tf.keys() if word in list(sentiment_dict.keys())]) 
    )

In [68]:
texts = [
    """
    Hvilke hestesteder har du været "tilknyttet" , og ca hvornår

    Her er en liste for mig

    Lejre Rideklub ( ca 1985 - 1990)
    Solrød Ridelub ( 1990 -1991 ) - red på en hest der stod opstaldet på den anden side af vejen
    Følfods rideklub ( 1991 -1995 )
    Gammeltoft rideklub ( 6 måneder sommer 1995)
    Adamslyst ridecenter ( 1996 - 1997)
    Baunehøj ridecenter ( red der ca 2 år hver anden uge i forbindelse med jeg gik På herlufsholm)
    Vallensbæk ridecenter ( 2002-2004)
    Dragør store magleby rideklub ( 2006 )

    PT
    Petersborg ( Høsterkøb) - Søster står der
    Og hjemme hos mig selv J

    Og så selvfølgelig Enghøjs ponyclub, hvor jeg er kommet siden 1985 og stadig kommer

    Jeg har godt nok holdt til Mange steder inden for de sidste 22 år J
    Man kan godt "læse" at jeg indtil nu udelukkende har været ½ parts rytter J

    Hvor har i holdt til??
    """,
    """
    Halløj.

    Nogen der kan anbefale et sted, hvor hesten kommer på enefold, da vi har brug for den kan være der 1 uge - 3 uger af gangen? Det skal være en radius af 50 km fra Ballerup. Boks er et plus, men løsdrift med fornuftigt læmulighed vil også fungere. Det er til en vallak på 145. Foder og træpiller kan medbringes, men brug for mulighed for at købe wrap.
    Dbh
    """,
    """
    
    Hej alle sammen.

    Jeg går med tanken om at skulle have min egen hest, men har ikke selv råd til at have en med alle udgifterne alene så derfor spørger jeg: er der nogen der har erfaring med det? Har prøvet det en gang og det gik fint, indtil jeg skulle flytte tilbage til Sjælland. Jeg er kun tur rytter og rider ture.

    Vh Tine
    """,
    """
    Jeg har været i gang med at lede efter et helt specifikt produkt - og blev simpelthen ramt af, hvor besværligt det er at søge noget som helst frem på Facebook.

    Prøvede så på HN, men her er søgningen begrænset til ét år tilbage (jeg mener, at den afgrænsning blev indført grundet mængden af data, man kan vist søge sen aften / tidlig morgen, eller hvad ved jeg).

    Hvor er det ærgerligt, at HN fik lov at dø. Tilsyneladende er der stadig reklameindtægter på siden, men den bliver simpelthen ikke udviklet længere.

    Er der egentlig nogen, der ved hvorfor? Det er jo stadig et site med et stort potentiale, men det er som det bare er blevet efterladt til en stille død. 
    """,
    """
    Halløj med jer :-)

    Vi skal til 50 års fødselsdag på lørdag. Min mand er pludselig kommet i tanke om en sjov sang, men kan selvfølgelig ikke huske teksten.. Den skulle starte meget fint, men så blive mere og mere mærkelig, med for lange og korte sætninger der ikke rimer og ikke kan synges ;-) Er der nogen af jer der kender den/ved hvor den kan findes? Jeg har prøvet google, men finder desværre kun frem til et link fra 2004 der ikke virker længere.

    På forhånd tak.
    """,
    """
    Hvilke Photo programmer bruger du ? Jeg ville gerne lave nogle flotte billeder med hesten, men ved ikke hvad jeg skal downloade. Hvad bruger i ?
    """,
    """
    Hej
    Jeg har en del hesteblad som Hestemagasinet, Ridehesten mm. som jeg ikke får læst. Jeg har overvejet at smide dem ud, men synes også det er synd. Tror I der er nogen som kunne være interesseret i dem, og hvor kan de sættes til salg?
    """
]

https://towardsdatascience.com/topic-modelling-in-python-with-nltk-and-gensim-4ef03213cd21

In [59]:
def clean_text(texts):
    return [
        [
            w for word in nltk.word_tokenize(text) 
            if not (len(word) < 4 or word in da_stop) 
            and (w:=lemmatizer.lemmatize(word).lower()).isalpha()
        ] 
        for text in texts
    ]

clt = clean_text(texts)
dictionary = corpora.Dictionary(clt)
corpus = [dictionary.doc2bow(text) for text in clt]

import gensim
NUM_TOPICS = 10
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=3)
print("\n".join([topic[1] for topic in topics]))

0.013*"rideklub" + 0.013*"ridecenter" + 0.013*"godt"
0.013*"brug" + 0.013*"boks" + 0.013*"anbefale"
0.062*"brug" + 0.033*"kommer" + 0.033*"nogen"
0.013*"brug" + 0.013*"boks" + 0.013*"anbefale"
0.013*"brug" + 0.013*"boks" + 0.013*"anbefale"
0.013*"brug" + 0.013*"boks" + 0.013*"anbefale"
0.013*"brug" + 0.013*"enefold" + 0.013*"medbringes"
0.013*"rideklub" + 0.013*"ridecenter" + 0.013*"holdt"
0.013*"brug" + 0.013*"rideklub" + 0.013*"vejen"
0.062*"rideklub" + 0.047*"ridecenter" + 0.032*"godt"


In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

def clean_text(texts):
    return [
        ' '.join(
            [
                w for word in nltk.word_tokenize(text) 
                if not (len(word) < 4 or word in da_stop) 
                and (w:=lemmatizer.lemmatize(word).lower()).isalpha()
            ]
        )
        for text in texts
    ]

document = clean_text(texts=texts)
# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_df=0.85, min_df=1, stop_words=list(da_stop))
tfidf_matrix = tfidf_vectorizer.fit_transform(document)

# Apply Non-Negative Matrix Factorization (NMF) for topic modeling
num_topics = 50  # You can adjust the number of topics as needed
nmf = NMF(n_components=num_topics, random_state=1)
nmf.fit(tfidf_matrix)

# Print the top words for each topic
feature_names = tfidf_vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(nmf.components_):
    top_words_idx = topic.argsort()[:-5:-1]  # Get the indices of the top 4 words for each topic
    top_words = [feature_names[i] for i in top_words_idx]
    print(', '.join(top_words))

kommet, selvfølgelig, prøvet, længere
hest, rytter, indtil, flytte
mere, tanke, huske, korte
ridehesten, hesteblad, overvejet, smide
søge, simpelthen, tilbage, stadig
søge, simpelthen, stadig, site
brug, læmulighed, ballerup, medbringes
rideklub, ridecenter, holdt, godt
hestemagasinet, synd, synes, smide
rideklub, ridecenter, godt, holdt
kommer, rideklub, gammeltoft, søster
bruger, lave, billeder, programmer
kommer, hesten, boks, anbefale
siden, stadig, aften, holdt
hesteblad, sættes, interesseret, salg
ridecenter, stod, udelukkende, selvfølgelig
fungere, wrap, brug, mulighed
simpelthen, morgen, indført, afgrænsning
tror, overvejet, synes, smide
fint, prøvet, ture, sammen
bruger, gerne, flotte, photo
hesten, kommer, løsdrift, anbefale
mere, finder, mærkelig, sang
selvfølgelig, kommet, forhånd, sjov
mere, virker, link, rimer
indtil, rytter, rider, derfor
hestesteder, godt, baunehøj, side
egen, udgifterne, tine, sammen
specifikt, grundet, hvorfor, besværligt
hesten, bruger, downloade, ph